# Format a latex table of reactions according to an input spreadsheet

In [1]:
import pandas as pd
import copy

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
main_folder = "/home/emc/OneDrive-CU/Research/UpperAtmoDH/Results/DGearI+iNT_bXwiopqk_smean_v13/"

# Useful function

This horrible block assigns reaction numbers with letters for multiple channels:

In [257]:
def format_spc(s):
    """
    Replace certain strings with latex strings
    """
    s = s.replace("pl", "^+")
    s = s.replace("E", "e^-")
    s = s.replace("2", "_2")
    s = s.replace("3", "_3")
    s = s.replace("up_2D", "(^2D)")
    s = s.replace("1D", "(^1D)")
    return s
    
def participant_str(r1, r2="", r3=""):
    
    # print(f"Working with {r1} and {r2}")
    if r1 != "":
        if (r2 != "") & (r2 != "none"):
            if (r3 != "") & (r3 != "none"):
                return f"{format_spc(r1)} + {format_spc(r2)} + {format_spc(r3)}"
            else:
                return f"{format_spc(r1)} + {format_spc(r2)}"
        else:
            return f"{format_spc(r1)}"
    else:
        return ""

def br_str(br):
    if (br == "") | (int(float(br)) == 1): # (int(float(br)) == 0) | 
        return "" 
    else:
        return f"{br}"

    
def f_str(f):
    if f == "":
        return "" 
    else:
        if float(f) != 0:
            return f"{f}"
        else: 
            return ""

    
def mscale_str(m2, m1, p):
    if m2 != 1:
        if p == -0.5:
            return f"$\sqrt{{ \\frac{{{m1}}}{{{m2}}} }}$"
        else:
            return f"$\left(\\frac{{{m2}}}{{{m1}}}\right)^{{{p}}}$"
    else:
        return ""
    
# Enter in the rate coefficient column
def a_str(a):
    Astring = str("{:.2e}".format(a))#Astring = str(a)
    if "e-" in Astring:
        Astring = Astring.replace("e-0", r" \times 10^{-")
        Astring = Astring.replace("e-", r" \times 10^{-")
        Astring = Astring + "}"

    return Astring

def b_str(b, T="T_i"):
    if b != 0:
        return f"\left(\\frac{{{T}}}{{300}}\right)^{{{b}}}"
    else:
        return ""

def c_str(c, T="T_i"):
    if c != 0:
        return f"e^{{{c}/{T}}}"
    else:
        return ""
    
def k_str(a, b, c, a0="", b0="", c0="", T="T_i"):
    if (a0 != 0) & (a0 != ""):
        return r"\makecell[l]{See text \\ k$_{\infty}="+f"{a_str(a)}{b_str(b, T=T)}{c_str(c, T=T)}$ \\\ k$_0={a_str(a0)}{b_str(b0, T=T)}{c_str(c0, T=T)}$" + r"}"
    else:
        return f"${a_str(a)}{b_str(b, T=T)}{c_str(c, T=T)}$"

def typestr(t):
    if t != "":
        if int(t) > 2:
            return str(int(t))
        else:
            return ""
    else:
        return ""

In [124]:
def a_str_new(a):

    Astring = str("{:.2e}".format(a))
    if "e-" in Astring:
        Astring = Astring.replace("e-0", r" \times 10^{-")
        Astring = Astring.replace("e-", r" \times 10^{-")
        Astring = Astring + "}"

    return Astring

a_str_new(1.851851851851e-30)

'1.85 \\times 10^{-30}'

In [90]:
def assign_reaction_numbers_old(df, division_ind=None):
    """
    division_ind: where it switches from unimolecular to bimolecular.
    """
    rxn_num = 1

    comparison_r1 = ""
    comparison_r2 = ""

    if division_ind != None:
        di=division_ind
    else:
        di=-2
        
    
    for index, row in df[0:di+1].iterrows():
        L = "a"

        # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

        # For when we don't have a repeated reactant set:
        if (comparison_r1 != df.loc[index, "R1"]) | ( ("R2" in df.columns) &  (comparison_r2 != df.loc[index, "R2"]) ):
            # print("No match")
            df.loc[index, "Num"] = rxn_num # give it a number 

            # This reaction becomes the point of comaprison
            comparison_r1 = df.loc[index, "R1"]
            comparison_r2 = df.loc[index, "R2"]
            # print(f"New comparisons {comparison_r1} and {comparison_r2}")

            # prepare to move on to the next
            rxn_num += 1
            comp_ind = index
            L = "a"
        elif (comparison_r1 == df.loc[index, "R1"]) & ( ("R2" in df.columns) & (comparison_r2 == df.loc[index, "R2"])):
            # print("matching")
            branchno = index - comp_ind + 1
            # print(f"Branch number is {branchno}")
            this_num = rxn_num -1
            # print(f"Reaction number to use is {this_num}")

            # Reletter the number of the first in the set with an a
            df.loc[comp_ind, "Num"] = str(this_num) + L 

            # Give this one a b, c, or whatever is appropriate
            df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

            # FIx up the reactant entries 
            df.loc[index, "R1"] = ""
            L = chr(ord(L)+1)

    if division_ind != None:
        for index, row in df[division_ind+1:].iterrows():
            L = "a"

            # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

            # For when we don't have a repeated reactant set:
            if (comparison_r1 != df.loc[index, "R1"]) or (("R2" in df.columns) & (comparison_r2 != df.loc[index, "R2"])):
                # print("No match")
                df.loc[index, "Num"] = rxn_num # give it a number 

                # This reaction becomes the point of comaprison
                comparison_r1 = df.loc[index, "R1"]
                comparison_r2 = df.loc[index, "R2"]
                # print(f"New comparisons {comparison_r1} and {comparison_r2}")

                # prepare to move on to the next
                rxn_num += 1
                comp_ind = index
                L = "a"
            elif (comparison_r1 == df.loc[index, "R1"]) and (("R2" in df.columns) & (comparison_r2 == df.loc[index, "R2"])):
                # print("matching")
                branchno = index - comp_ind + 1
                # print(f"Branch number is {branchno}")
                this_num = rxn_num -1
                # print(f"Reaction number to use is {this_num}")

                # Reletter the number of the first in the set with an a
                df.loc[comp_ind, "Num"] = str(this_num) + L 

                # Give this one a b, c, or whatever is appropriate
                df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

                # FIx up the reactant entries 
                df.loc[index, "R1"] = ""
                df.loc[index, "R2"] = ""
                L = chr(ord(L)+1)
            # print()
            
def assign_reaction_numbers(df, opt_start=1):
    """
    df: dataframe
    
    opt_start: optional number at which to start the reaction numbering, useful for stitching tables together with multiple calls
    """
    
    rxn_num = opt_start

    comp = {"R1": "", "R2": "", "R3": ""}

    di=-2
        
    # Identify which columns are present in this dataframe
    rcols = set(comp.keys()).intersection(df.columns) 
    
    for index, row in df.iterrows():
        L = "a"

        # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

        reactants_this_row = [df.loc[index, r] for r in rcols]
        this_row_str = "+".join(reactants_this_row)
        to_compare = [comp[r] for r in rcols]
        join_compare = "+".join(to_compare)
        
        # print(f"comparing {to_compare} and {reactants_this_row}")

        # For when we don't have a repeated reactant set:
        if to_compare != reactants_this_row: 
            # print(f"row {index} not a match: {join_compare} != {this_row_str}")
            df.loc[index, "Num"] = rxn_num # give it a number 

            # This reaction becomes the point of comaprison: Update the dictionary with its reactant values
            for r in rcols:
                comp[r] = df.loc[index, r]

            # print(f"New comparisons {comparison_r1} and {comparison_r2}")

            # prepare to move on to the next
            rxn_num += 1
            comp_ind = index
            L = "a"
        else: # For when the reactants all match, i.e. we are looking at another branch:
            # print(f"row {index} match: {join_compare} = {this_row_str}")
            branchno = index - comp_ind + 1
            # print(f"Branch number is {branchno}")
            this_num = rxn_num - 1
            # print(f"Reaction number to use is {this_num}")

            # Reletter the number of the first in the set with an a
            df.loc[comp_ind, "Num"] = str(this_num) + L 

            # Give this one a b, c, or whatever is appropriate
            df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

            # Clear out the reactants because we don't need them printed again
            for r in rcols:
                df.loc[index, r] = ""
                
            L = chr(ord(L)+1)

    

# Load

In [230]:
# We read the excel file as a dataframe
df_photod = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photodissociation", 
                    index_col=None, usecols=["R1", "P1", "P2", "BR", "Reference"])

df_photoi = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photoionization", 
                    index_col=None, usecols=["R1", "P1", "P2", "BR", "Reference"])

df_neutrals = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Neutral reactions", 
                    index_col=None, usecols=["R1", "R2", "R3", "P1", "P2", "P3", "type", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "F", "Reference"])

df_ions = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Ion reactions", 
                    index_col=None, usecols=["R1", "R2", "P1", "P2", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "Reference"])



Look for only deuterium reactions, and add some columns to hold the reaction number and some latex to add hspace, then reset the index so the df isn't messed up:

In [115]:
df_photod_D = df_photod[ df_photod["R1"].str.contains("D") ]
df_photoi_D = df_photoi[ df_photoi["R1"].str.contains("D") ]

# Separate



## Photod + i

In [116]:
photo_df = pd.concat([df_photod_D, df_photoi_D], ignore_index=True)

# Blank out any nans for R2
photo_df[['BR']] = photo_df[['BR']].fillna("")
photo_df[['Reference']] = photo_df[['Reference']].fillna("")
photo_df['P2'] = photo_df['P2'].replace(to_replace="none", value='')

# Add reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)
photo_df.loc[:, "Rate"] = ""
photo_df.reset_index(inplace=True)
photo_df = photo_df.drop(["index"], axis=1) # don't need this column

# Initial sort
photo_df.sort_values(["R1", "P1", "P2"], inplace=True, ignore_index=True)

# Manual sort to group by reactant 1
d = {2: 3, 3: 2, 4: 5, 5: 4, 
     7: 10, 9:7, 10:9, 11:12, 12:11}
photo_df = photo_df.rename(d).sort_index()

# Re-do the reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)


In [117]:
# Assign reaction numbers
assign_reaction_numbers(photo_df)

print(photo_df)

# Make the strings 
photo_df = photo_df.assign(reactants = [f"$\mathrm{{{participant_str(r1)}}}$" if r1!="" else "" for r1 in photo_df["R1"]] )
photo_df = photo_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2)}}}$" for (p1,p2) in zip(photo_df["P1"], photo_df["P2"])])

# Make the final df
photodf_final = photo_df.drop(["R1", "P1", "P2"], axis=1)
photodf_final = photodf_final[["Num", "reactants", "products", "Rate", "Reference"]]

# We save the dataframe into a latex table
photodf_final.style.format(escape="latex")  
photodf_final.to_latex("ReactionTables/photo_reactions.tex", index=False, label="photo_rxns", longtable=True, escape=False)

      R1     P1   P2   BR Reference Num Rate
0      D    Dpl                       1     
1    DO2     OD    O                  2     
2     HD      H    D                 3a     
3           Dpl    H  0.5            3b     
4           Hpl    D  0.5            3c     
5          HDpl                      3d     
6    HDO      D   OH                 4a     
7             H   OD                 4b     
8             H    D                 4c     
9            HD  O1D                 4d     
10          Dpl   OH  0.5            4e     
11          Hpl   OD  0.5            4f     
12        HDOpl                      4g     
13         ODpl    H  0.5            4h     
14         OHpl    D  0.5            4i     
15          Opl   HD                 4j     
16  HDO2    DO2    H                 5a     
17          HDO  O1D                 5b     
18          HO2    D                 5c     
19           OH   OD                 5d     
20    OD      O    D                 6a     
21        

/tmp/ipykernel_11098/1425723958.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  photodf_final.to_latex("ReactionTables/photo_reactions.tex", index=False, label="photo_rxns", longtable=True, escape=False)


In [82]:
photodf_final


,Num,reactants,products,BR,Reference
0,1,$\mathrm{D}$,$\mathrm{D^+}$,,
1,2,$\mathrm{DO_2}$,$\mathrm{OD + O}$,,
2,3a,$\mathrm{HD}$,$\mathrm{H + D}$,,
3,3b,$\mathrm{}$,$\mathrm{D^+ + H}$,0.5,
4,3c,$\mathrm{}$,$\mathrm{H^+ + D}$,0.5,
5,3d,$\mathrm{}$,$\mathrm{HD^+}$,,
6,4a,$\mathrm{HDO}$,$\mathrm{D + OH}$,,
7,4b,$\mathrm{}$,$\mathrm{H + OD}$,,
8,4c,$\mathrm{}$,$\mathrm{H + D}$,,
9,4d,$\mathrm{}$,$\mathrm{HD + O(^1D)}$,,


## Neutrals

In [220]:
all_species = ["Ar", "CO", "CO2", "H", "H2", "H2O", "H2O2", "HO2", "HOCO", "N2", "O", "O1D", "O2", "O3", "OH","D", "DO2", "DOCO", "HD", "HDO", "HDO2", "OD","C", "CH", "CN", "DCO", "HCN", "HCO", "HNO", "N", "NH", "NH2", "N2O", "NO", "NO2", "Nup2D","CO2pl", "HCO2pl", "Opl", "O2pl",  "Arpl", "ArHpl", "ArDpl", "Cpl", "CHpl", "CNpl", "COpl", "Hpl", "Dpl", "H2pl", "HDpl", "H3pl", "H2Dpl", "HD2pl", "H2Opl",  "HDOpl", "H3Opl", "H2DOpl", "HO2pl", "HCOpl", "DCOpl", "HOCpl", "DOCpl", "DCO2pl", "HCNpl", "HCNHpl", "HNOpl", "HN2Opl",  "Npl",  "NHpl", "NH2pl", "NH3pl", "N2pl", "N2Hpl", "N2Dpl", "N2Opl", "NOpl", "NO2pl","OHpl", "ODpl"]

In [258]:
D_species = [a  for a in all_species if (("D" in a) & (a not in ["O1D", "Nup2D"]))]
print(len(D_species))

20


In [259]:
df_neutrals_D = df_neutrals[ df_neutrals["R1"].isin(D_species) | df_neutrals["R2"].isin(D_species) | df_neutrals["R3"].isin(D_species) ]


In [260]:
n_df = copy.deepcopy(df_neutrals_D)

# Blank out any nans
n_df.loc[:, 'R2'] = n_df.loc[:, 'R2'].fillna("")
n_df.loc[:, 'R3'] = n_df.loc[:, 'R3'].fillna("")
n_df.loc[:, 'P2'] = n_df.loc[:, 'P2'].fillna("")
n_df.loc[:, 'P3'] = n_df.loc[:, 'P3'].fillna("")
n_df.loc[:, 'type'] = n_df.loc[:, 'type'].fillna("")
n_df.loc[:, 'M2'] = n_df.loc[:, 'M2'].fillna("")
n_df.loc[:, 'M1'] = n_df.loc[:, 'M1'].fillna("")
n_df.loc[:, 'pow'] = n_df.loc[:, 'pow'].fillna("")
n_df.loc[:, 'BR'] = n_df.loc[:, 'BR'].fillna("")
n_df.loc[:, 'kA'] = n_df.loc[:, 'kA'].fillna("")
n_df.loc[:, 'kB'] = n_df.loc[:, 'kB'].fillna("")
n_df.loc[:, 'kC'] = n_df.loc[:, 'kC'].fillna("")
n_df.loc[:, 'k0A'] = n_df.loc[:, 'k0A'].fillna("")
n_df.loc[:, 'k0B'] = n_df.loc[:, 'k0B'].fillna("")
n_df.loc[:, 'k0C'] = n_df.loc[:, 'k0C'].fillna("")
n_df.loc[:, 'F'] = n_df.loc[:, 'F'].fillna("")


n_df.sort_values(["R1", "R2", "R3", "P1", "P2"], inplace=True, ignore_index=True)

In [261]:
# Add reaction number column
n_df.loc[:, "Num"] = range(1, len(n_df)+1)
n_df.reset_index(inplace=True)
n_df = n_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(n_df, opt_start=7)

In [262]:
n_df

,R1,R2,R3,P1,P2,P3,type,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,F,Reference,Num
0,CO,D,none,DCO,none,none,4,1,1,0.0,1.0,1.000000e+00,0.20,0.0,2.000000e-35,0.2,0,0.0,Baulch05 (H analogue rate),7
1,CO,OD,none,CO2,D,none,6,18,17,-0.5,1.0,1.628470e-06,6.10,0.0,4.895742e-15,0.6,0,0.0,Mass scaling,8a
2,,,,DOCO,none,none,5,18,17,-0.5,1.0,6.624225e-16,1.30,0.0,1.733093e-29,-1.4,0,0.0,Mass scaling,8b
3,D,H2,none,HD,H,none,2,1,1,0.0,1.0,2.725102e-17,2.00,-2700.0,0.000000e+00,0.0,0,0.0,NIST (Mitchell & Le Roy 1973),9
4,D,H2O2,none,H2O,OD,none,2,1,1,0.0,0.5,1.160000e-11,0.00,-2110.0,0.000000e+00,0.0,0,0.0,Cazaux2010,10a
5,,,,HDO,OH,none,2,1,1,0.0,0.5,1.160000e-11,0.00,-2110.0,0.000000e+00,0.0,0,0.0,Cazaux2010,10b
6,D,HO2,none,DO2,H,none,2,1,1,0.0,1.0,1.000000e-10,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988,11a
7,,,,HD,O2,none,2,1,1,0.0,1.0,2.449500e-12,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11b
8,,,,HDO,O1D,none,2,1,1,0.0,1.0,1.136000e-12,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11c
9,,,,OH,OD,none,2,1,1,0.0,1.0,5.112000e-11,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11d


In [263]:
# Make the strings 
n_df = n_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2,r3)}}}$" if r1!="" else "" for (r1,r2,r3) in zip(n_df["R1"], n_df["R2"], n_df["R3"])])
n_df = n_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2,p3)}}}$" if p1!="" else ""  for (p1,p2,p3) in zip(n_df["P1"], n_df["P2"], n_df["P3"])])

# Other column strings
n_df = n_df.assign(ratecoeff = [k_str(A,B,C,a0=A0,b0=B0,c0=C0,T="T_n") for (A,B,C,A0,B0,C0) in zip(n_df["kA"], n_df["kB"], n_df["kC"], n_df["k0A"], n_df["k0B"], n_df["k0C"])])
n_df = n_df.assign(type = [f"{typestr(t)}" for t in n_df["type"]])
n_df = n_df.assign(BR = [f"{br_str(BR)}" for BR in n_df["BR"]])
n_df = n_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(n_df["M2"], n_df["M1"], n_df["pow"])])
# n_df = n_df.assign(F = [f"{f_str(f)}" for f in n_df["F"]]) # No D reactions have troe parameter.

# Make the final df
n_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
ndf_final = n_df.drop(["R1", "R2", "R3", "P1", "P2", "P3", "M2", "M1", "pow",  "kA", "kB", "kC", "k0A", "k0B", "k0C"], axis=1)
ndf_final = ndf_final[["Num", "reactants", "products", "BR", "MS", "Rate coefficient", "Reference"]]

# We save the dataframe into a latex table
ndf_final.style.format(escape="latex")  
ndf_final.to_latex("ReactionTables/neutral_reactions.tex", index=False, label="neutral_rxns", longtable=True, escape=False)

/tmp/ipykernel_11098/3355195997.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ndf_final.to_latex("ReactionTables/neutral_reactions.tex", index=False, label="neutral_rxns", longtable=True, escape=False)


In [256]:
ndf_final[0:30]

,Num,reactants,products,BR,MS,Rate coefficient,Reference
0,7,$\mathrm{CO + D}$,$\mathrm{DCO}$,,,\makecell[l]{k$_{\infty}=1.00e+00\left(\frac{T_n}{300}\right)^{0.2}$ \\ k$_0=2.00 \times 10^{-35}\left(\frac{T_n}{300}\right)^{0.2}$},Baulch05 (H analogue rate)
1,8a,$\mathrm{CO + OD}$,$\mathrm{CO_2 + D}$,,$\sqrt{ \frac{17}{18} }$,\makecell[l]{k$_{\infty}=1.63 \times 10^{-6}\left(\frac{T_n}{300}\right)^{6.1}$ \\ k$_0=4.90 \times 10^{-15}\left(\frac{T_n}{300}\right)^{0.6}$},Mass scaling
2,8b,,$\mathrm{DOCO}$,,$\sqrt{ \frac{17}{18} }$,\makecell[l]{k$_{\infty}=6.62 \times 10^{-16}\left(\frac{T_n}{300}\right)^{1.3}$ \\ k$_0=1.73 \times 10^{-29}\left(\frac{T_n}{300}\right)^{-1.4}$},Mass scaling
3,9,$\mathrm{D + H_2}$,$\mathrm{HD + H}$,,,$2.73 \times 10^{-17}\left(\frac{T_n}{300}\right)^{2.0}e^{-2700.0/T_n}$,NIST (Mitchell & Le Roy 1973)
4,10a,$\mathrm{D + H_2O_2}$,$\mathrm{H_2O + OD}$,0.5,,$1.16 \times 10^{-11}e^{-2110.0/T_n}$,Cazaux2010
5,10b,,$\mathrm{HDO + OH}$,0.5,,$1.16 \times 10^{-11}e^{-2110.0/T_n}$,Cazaux2010
6,11a,$\mathrm{D + HO_2}$,$\mathrm{DO_2 + H}$,,,$1.00 \times 10^{-10}$,Yung1988
7,11b,,$\mathrm{HD + O_2}$,,,$2.45 \times 10^{-12}$,Yung1988: 0.71xH-analogue
8,11c,,$\mathrm{HDO + O(^1D)}$,,,$1.14 \times 10^{-12}$,Yung1988: 0.71xH-analogue
9,11d,,$\mathrm{OH + OD}$,,,$5.11 \times 10^{-11}$,Yung1988: 0.71xH-analogue


## Ions

In [275]:
df_ions_D = df_ions[ df_ions["R1"].isin(D_species) | df_ions["R2"].isin(D_species) ]

In [276]:
i_df = copy.deepcopy(df_ions_D)

# Blank out any nans for R2
i_df.loc[:, 'R2'] = i_df.loc[:, 'R2'].fillna("")
i_df.loc[:, 'P2'] = i_df.loc[:, 'P2'].fillna("")
i_df.loc[:, 'M2'] = i_df.loc[:, 'M2'].fillna("")
i_df.loc[:, 'M1'] = i_df.loc[:, 'M1'].fillna("")
i_df.loc[:, 'pow'] = i_df.loc[:, 'pow'].fillna("")
i_df.loc[:, 'BR'] = i_df.loc[:, 'BR'].fillna("")
i_df.loc[:, 'kA'] = i_df.loc[:, 'kA'].fillna("")
i_df.loc[:, 'kB'] = i_df.loc[:, 'kB'].fillna("")
i_df.loc[:, 'kC'] = i_df.loc[:, 'kC'].fillna("")

i_df.sort_values(["R1", "R2", "P1", "P2"], inplace=True, ignore_index=True)

i_df

,R1,R2,P1,P2,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,Reference
0,ArDpl,CO,DCOpl,Ar,1,1,0.0,1.00,1.250000e-09,0.0,0.0,0,0,0,Anicich2003
1,ArDpl,CO2,DCO2pl,Ar,1,1,0.0,1.00,1.100000e-09,0.0,0.0,0,0,0,Anicich2003
2,ArDpl,H2,ArHpl,HD,1,1,0.0,1.00,4.500000e-10,0.0,0.0,0,0,0,Anicich2003
3,ArDpl,H2,H2Dpl,Ar,1,1,0.0,1.00,8.800000e-10,0.0,0.0,0,0,0,Anicich2003
4,ArDpl,N2,N2Dpl,Ar,1,1,0.0,1.00,6.000000e-10,0.0,0.0,0,0,0,Anicich2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,OHpl,HD,HDOpl,H,3,2,-0.5,1.00,9.700000e-10,0.0,0.0,0,0,0,Mass scaling
121,Opl,D,Dpl,O,2,1,-0.5,1.00,6.400000e-10,0.0,0.0,0,0,0,Mass scaling
122,Opl,HD,ODpl,H,1,1,0.0,0.46,1.250000e-09,0.0,0.0,0,0,0,Anicich2003
123,Opl,HD,OHpl,D,1,1,0.0,0.54,1.250000e-09,0.0,0.0,0,0,0,Anicich2003


In [277]:
# Add reaction number column
i_df.loc[:, "Num"] = range(1, len(i_df)+1)
i_df.reset_index(inplace=True)
i_df = i_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(i_df, opt_start=50)

In [278]:
i_df

,R1,R2,P1,P2,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,Reference,Num
0,ArDpl,CO,DCOpl,Ar,1,1,0.0,1.00,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,50
1,ArDpl,CO2,DCO2pl,Ar,1,1,0.0,1.00,1.100000e-09,0.0,0.0,0,0,0,Anicich2003,51
2,ArDpl,H2,ArHpl,HD,1,1,0.0,1.00,4.500000e-10,0.0,0.0,0,0,0,Anicich2003,52a
3,,,H2Dpl,Ar,1,1,0.0,1.00,8.800000e-10,0.0,0.0,0,0,0,Anicich2003,52b
4,ArDpl,N2,N2Dpl,Ar,1,1,0.0,1.00,6.000000e-10,0.0,0.0,0,0,0,Anicich2003,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,,,HDOpl,H,3,2,-0.5,1.00,9.700000e-10,0.0,0.0,0,0,0,Mass scaling,122b
121,Opl,D,Dpl,O,2,1,-0.5,1.00,6.400000e-10,0.0,0.0,0,0,0,Mass scaling,123
122,Opl,HD,ODpl,H,1,1,0.0,0.46,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,124a
123,,,OHpl,D,1,1,0.0,0.54,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,124b


In [279]:
# Make the strings 
i_df = i_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2)}}}$" for (r1,r2) in zip(i_df["R1"], i_df["R2"])])
i_df = i_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2)}}}$" for (p1,p2) in zip(i_df["P1"], i_df["P2"])])

# Other column strings
i_df = i_df.assign(ratecoeff = [k_str(A,B,C) for (A,B,C) in zip(i_df["kA"], i_df["kB"], i_df["kC"])])
i_df = i_df.assign(BR = [f"{br_str(BR)}" for BR in i_df["BR"]])
i_df = i_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(i_df["M2"], i_df["M1"], i_df["pow"])])

# Make the final df
i_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
idf_final = i_df.drop(["R1", "R2", "P1", "P2", "M2", "M1", "pow",  "kA", "kB", "kC"], axis=1)
idf_final = idf_final[["Num", "reactants", "products",  "BR", "MS", "Rate coefficient", "Reference"]]

# We save the dataframe into a latex table
idf_final.style.format(escape="latex")  
idf_final.to_latex("ReactionTables/ion_reactions.tex", index=False, label="ion_rxns", longtable=True, escape=False)

/tmp/ipykernel_11098/3073714054.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  idf_final.to_latex("ReactionTables/ion_reactions.tex", index=False, label="ion_rxns", longtable=True, escape=False)


# Altogether

In [6]:
big_df = pd.concat([df_photod_D, df_photoi_D, df_neutrals_D, df_ions_D], ignore_index=True)
big_df

,R1,P1,P2,M2,M1,pow,BR,Reference,R2,R3,P3,type,kA,kB,kC,k0A,k0B,k0C,F
0,DO2,OD,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HD,H,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HDO,D,OH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HDO,H,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HDO,H,OD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,HDOpl,O,D,19.0,18.0,-0.5,1.0,Mass scaling,E,NaN,NaN,NaN,2.080000e-05,-0.740,0.0,NaN,NaN,NaN,NaN
225,HDOpl,OH,D,19.0,18.0,-0.5,0.5,Mass scaling,E,NaN,NaN,NaN,5.860000e-06,-0.740,0.0,NaN,NaN,NaN,NaN
226,HDpl,H,D,1.0,1.0,0.0,1.0,KIDA,E,NaN,NaN,NaN,1.932730e-06,-0.853,-43.3,NaN,NaN,NaN,NaN
227,N2Dpl,N2,D,30.0,29.0,-0.5,1.0,Mass scaling,E,NaN,NaN,NaN,6.600000e-07,-0.510,0.0,NaN,NaN,NaN,NaN


In [7]:
# Blank out any nans for R2
big_df[['R2']] = big_df[['R2']].fillna("")
big_df[['R3']] = big_df[['R3']].fillna("")
big_df[['P2']] = big_df[['P2']].fillna("")
big_df[['P3']] = big_df[['P3']].fillna("")
big_df[['type']] = big_df[['type']].fillna("")
big_df[['M2']] = big_df[['M2']].fillna("")
big_df[['M1']] = big_df[['M1']].fillna("")
big_df[['pow']] = big_df[['pow']].fillna("")
big_df[['BR']] = big_df[['BR']].fillna("")
big_df[['kA']] = big_df[['kA']].fillna("")
big_df[['kB']] = big_df[['kB']].fillna("")
big_df[['kC']] = big_df[['kC']].fillna("")
big_df[['k0A']] = big_df[['k0A']].fillna("")
big_df[['k0B']] = big_df[['k0B']].fillna("")
big_df[['k0C']] = big_df[['k0C']].fillna("")
big_df[['F']] = big_df[['F']].fillna("")

In [8]:
# Add a rxn label col       
big_df.loc[:, "Num"] = range(1, len(big_df)+1)

big_df.reset_index(inplace=True)
big_df = big_df.drop(["index"], axis=1)

In [9]:
big_df[0:30]

,R1,P1,P2,M2,M1,pow,BR,Reference,R2,R3,P3,type,kA,kB,kC,k0A,k0B,k0C,F,Num
0,DO2,OD,O,,,,,NaN,,,,,,,,,,,,1
1,HD,H,D,,,,,NaN,,,,,,,,,,,,2
2,HDO,D,OH,,,,,NaN,,,,,,,,,,,,3
3,HDO,H,D,,,,,NaN,,,,,,,,,,,,4
4,HDO,H,OD,,,,,NaN,,,,,,,,,,,,5
5,HDO,HD,O1D,,,,,NaN,,,,,,,,,,,,6
6,HDO2,DO2,H,,,,,NaN,,,,,,,,,,,,7
7,HDO2,HDO,O1D,,,,,NaN,,,,,,,,,,,,8
8,HDO2,HO2,D,,,,,NaN,,,,,,,,,,,,9
9,HDO2,OH,OD,,,,,NaN,,,,,,,,,,,,10


In [11]:
assign_reaction_numbers(big_df, 25)

In [12]:
big_df[0:147]

,R1,P1,P2,M2,M1,pow,BR,Reference,R2,R3,P3,type,kA,kB,kC,k0A,k0B,k0C,F,Num
0,DO2,OD,O,,,,,NaN,,,,,,,,,,,,1
1,HD,H,D,,,,,NaN,,,,,,,,,,,,2
2,HDO,D,OH,,,,,NaN,,,,,,,,,,,,3a
3,,H,D,,,,,NaN,,,,,,,,,,,,3b
4,,H,OD,,,,,NaN,,,,,,,,,,,,3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Dpl,DCOpl,O,1.0,1.0,0.0,1.0,Anicich2003,CO2,,,,0.0,0.0,0.0,,,,,95a
143,,CO2pl,D,1.0,1.0,0.0,1.0,Anicich2003,,,,,0.0,0.0,0.0,,,,,95b
144,Dpl,D,Hpl,1.0,1.0,0.0,0.87,Yung1989,H,,,,0.0,0.5,0.0,,,,,96
145,Dpl,Hpl,HD,1.0,1.0,0.0,1.0,Anicich2003,H2,,,,0.0,0.0,0.0,,,,,97


In [14]:
# Assign reaction, rate coefficient, BR, and MS column 
big_df = big_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2,r3)}}}$" for (r1,r2,r3) in zip(big_df["R1"], big_df["R2"], big_df["R3"])])
big_df = big_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2,p3)}}}$" for (p1,p2,p3) in zip(big_df["P1"], big_df["P2"], big_df["P3"])])

# Handle some reactions only having kinf and some having k0
big_df = big_df.assign(ratecoeff = [k_str(A,B,C,A0,B0,C0) for (A,B,C,A0,B0,C0) in zip(big_df["kA"], big_df["kB"], big_df["kC"], big_df["k0A"], big_df["k0B"], big_df["k0C"])])
big_df = big_df.assign(type = [f"{typestr(t)}" for t in big_df["type"]])
big_df = big_df.assign(BR = [f"{br_str(BR)}" for BR in big_df["BR"]])
big_df = big_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(big_df["M2"], big_df["M1"], big_df["pow"])])
big_df = big_df.assign(F = [f"{f_str(f)}" for f in big_df["F"]])

In [15]:
big_df[95:102]

,R1,P1,P2,M2,M1,pow,BR,Reference,R2,R3,...,kC,k0A,k0B,k0C,F,Num,reactants,products,ratecoeff,MS
95,OH,HDO,H,1.0,1.0,0.0,1.0,Sander2011,HD,none,...,-2130.0,0.0,0.0,0.0,0.0,67a,$\mathrm{OH + HD}$,$\mathrm{HDO + H}$,$5 \times 10^{-12}e^{-2130.0/T_i}$,
96,,H2O,D,1.0,1.0,0.0,1.0,Yung1988,,none,...,-1800.0,0.0,0.0,0.0,0.0,67b,$\mathrm{}$,$\mathrm{H_2O + D}$,$4.2 \times 10^{-13}e^{-1800.0/T_i}$,
97,OH,HDO,HO2,35.0,34.0,-0.5,0.5,Mass scaling,HDO2,none,...,-160.0,0.0,0.0,0.0,0.0,68a,$\mathrm{OH + HDO_2}$,$\mathrm{HDO + HO_2}$,$2.9 \times 10^{-12}e^{-160.0/T_i}$,$\left(\frac{35.0}{34.0}\right)^{-0.5}$
98,,H2O,DO2,35.0,34.0,-0.5,0.5,Mass scaling,,none,...,-160.0,0.0,0.0,0.0,0.0,68b,$\mathrm{}$,$\mathrm{H_2O + DO_2}$,$2.9 \times 10^{-12}e^{-160.0/T_i}$,$\left(\frac{35.0}{34.0}\right)^{-0.5}$
99,CO,DCO,none,1.0,1.0,0.0,1.0,Baulch05 (H analogue rate),D,none,...,0.0,0.0,0.2,0.0,0.0,69,$\mathrm{CO + D}$,$\mathrm{DCO}$,"k$_{inf}=$$1.0\left(\frac{T_i}{300}\right)^{0.2}$, k$_0=2 \times 10^{-35}\left(\frac{T_i}{300}\right)^{0.2}$",
100,CO,DOCO,none,18.0,17.0,-0.5,1.0,Mass scaling,OD,none,...,0.0,0.0,-1.4,0.0,0.0,70,$\mathrm{CO + OD}$,$\mathrm{DOCO}$,"k$_{inf}=$$6.62422467428728 \times 10^{-16}\left(\frac{T_i}{300}\right)^{1.3}$, k$_0=1.73309260137893 \times 10^{-29}\left(\frac{T_i}{300}\right)^{-1.4}$",$\left(\frac{18.0}{17.0}\right)^{-0.5}$
101,D,DO2,none,2.0,1.0,-0.5,1.0,Mass scaling,O2,none,...,0.0,0.0,-1.3,0.0,0.0,71,$\mathrm{D + O_2}$,$\mathrm{DO_2}$,"k$_{inf}=$$2.39682878878546 \times 10^{-11}\left(\frac{T_i}{300}\right)^{0.2}$, k$_0=1.46130309220551 \times 10^{-28}\left(\frac{T_i}{300}\right)^{-1.3}$",$\left(\frac{2.0}{1.0}\right)^{-0.5}$


In [16]:
big_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
big_df_final = big_df.drop(["R1", "R2", "R3", "P1", "P2", "P3", "M2", "M1", "pow",  "kA", "kB", "kC", "k0A", "k0B", "k0C"], axis=1)
big_df_final = big_df_final[["Num", "reactants", "products",  "type", "BR", "MS", "Rate coefficient", "Reference"]]

In [18]:
big_df_final[0:40]

,Num,reactants,products,type,BR,MS,Rate coefficient,Reference
0,1,$\mathrm{DO_2}$,$\mathrm{OD + O}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
1,2,$\mathrm{HD}$,$\mathrm{H + D}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
2,3a,$\mathrm{HDO}$,$\mathrm{D + OH}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
3,3b,$\mathrm{}$,$\mathrm{H + D}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
4,3c,$\mathrm{}$,$\mathrm{H + OD}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
5,3d,$\mathrm{}$,$\mathrm{HD + O(^1D)}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
6,4a,$\mathrm{HDO_2}$,$\mathrm{DO_2 + H}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
7,4b,$\mathrm{}$,$\mathrm{HDO + O(^1D)}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
8,4c,$\mathrm{}$,$\mathrm{HO_2 + D}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN
9,4d,$\mathrm{}$,$\mathrm{OH + OD}$,,,$\left(\frac{}{}\right)^{}$,$\left(\frac{T_i}{300}\right)^{}e^{/T_i}$,NaN


In [19]:
big_df_final.loc[0:23, "Rate coefficient"] = ""

In [20]:
# We save the dataframe into a latex table
big_df_final.style.format(escape="latex")  
big_df_final.to_latex("ReactionTables/D_table_with_neutrals.tex", index=False, label="reaction_network", longtable=True, escape=False)

/tmp/ipykernel_22730/1935358734.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  big_df_final.to_latex("ReactionTables/D_table_with_neutrals.tex", index=False, label="reaction_network", longtable=True, escape=False)
